In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

dosya_yolu = 'turkish_tv_series.xlsx' 
df = pd.read_excel(dosya_yolu)


In [ ]:
# Yeterli bilgisi olmayan satırların silinmesi
df = df[df['Dizi İsimleri'] != 0]
df = df.reset_index(drop=True)

# gösterim süresinin int e cevirilmesi ve boş değerlerin platform ortalaması ile doldurulması
df['Gösterim Süresi'] = df['Gösterim Süresi'].replace(0, '0 dakika')
df['Gösterim Süresi'] = df['Gösterim Süresi'].str.replace('dakika','')
df['Gösterim Süresi'] = df['Gösterim Süresi'].str.replace('Dakika','')
df['Gösterim Süresi'] = df['Gösterim Süresi'].apply(lambda  x: x.strip()).astype(int)
platform_ortalamalari = df.groupby('Platform')['Gösterim Süresi'].mean().apply(np.ceil).astype(int).to_dict()
for index, row in df.iterrows():
    if row['Gösterim Süresi'] == 0:
        platform = row['Platform']
        ortalama = platform_ortalamalari[platform]
        df.at[index, 'Gösterim Süresi'] = ortalama

In [ ]:
#aktörlerin basrol oldugu dizi sayısı bulunuyor ve ortalaması alınıyor
actor_counts = defaultdict(int)

for actors in df['Başroller']:
    actor_list = actors.split(',')
    for actor in actor_list:
        actor_counts[actor] += 1

df['Aktör Etkisi'] = df['Başroller'].apply(lambda x: sum(actor_counts[actor.strip()] for actor in x.split(',')) / len(x.split(',')))

In [ ]:
with open('directors.txt', 'r') as dosya:
    yonetmenler = dosya.readlines()

yonetmenler = [yonetmen.strip() for yonetmen in yonetmenler]

df['Award Winning Director'] = df['Yönetmen'].apply(lambda x: 1 if x in yonetmenler else 0)


with open('actors.txt', 'r') as dosya2:
    oyuncular = dosya2.readlines()

oyuncular = [oyuncu.strip() for oyuncu in oyuncular]

df['Award Winning Actor'] = df['Başroller'].apply(lambda x: 1 if any(oyunc in oyuncular for oyunc in x.split(',')) else 0)

In [ ]:
for index, row in df.iterrows():
    if row['Sezon Sayısı'] == 1:
        df.loc[index, 'Sonuç'] = 'Başarısız'
    elif row['Sezon Sayısı'] == 2:
        df.loc[index, 'Sonuç'] = 'Kısmen Başarılı'
    elif row['Sezon Sayısı'] < 5:
        df.loc[index, 'Sonuç'] = 'Başarılı'
    elif row['Sezon Sayısı'] < 10:
        df.loc[index, 'Sonuç'] = 'Kısmen Olgun'
    elif row['Sezon Sayısı'] >= 10:
        df.loc[index, 'Sonuç'] = 'Olgun'

In [ ]:
df.drop('Türler', axis=1, inplace=True)
df.drop('Sezon Sayısı', axis=1, inplace=True)
df.drop('Sezondaki Ortalama Bölüm Sayısı', axis=1, inplace=True)

In [ ]:
df = df[df['Award Winning Actor'] == 1]
df.head(34)

In [ ]:
'''filtre = df['Yönetmen'] == 'Çağan Irmak'
gosterim_sayilari = df[filtre]['Gösterim Süresi']
print(gosterim_sayilari)'''

In [ ]:
'''filtre = df['Sezon Sayısı'] > 3
gosterim_sayilari = df[filtre]['Yönetmen']
print(gosterim_sayilari)'''

In [ ]:
#en_cok_tekrar_eden = df['Sezon Sayısı'].value_counts()
##print(en_cok_tekrar_eden)

In [ ]:
award_winnnig_director = ['Osman Sınav','Nihat Durak','Çağatay Tosun','Uluç Bayraktar','Cem Karcı','Altan Dönmez','Ali Bilgin','Çağrı Vila Lostuvalı','Yağız Alp Akaydın','Emre Kabakuşak','Yusuf Pirhasan']

award_winning_actors = ['Kadir İnanır','Mehmet Ali Alabora','Haluk Bilginer','Tamer Karadağlı','Mehmet Aslantuğ','Özcan Deniz','Timuçin Esen','Halit Ergenç','Halil Ergün','Kıvanç Tatlıtuğ','Kenan İmirzalıoğlu','Erkan Petekkaya','Çağatay Ulusoy','İlker Kaleli','Tolga Sarıtaş','Aras Bulut İynemli','Taner Ölmez','Çağlar Ertuğrul','Kaan Urgancıoğlu','Salih Bademci','Hande Ataizi','Meltem Cumbul','Türkan Şoray','Gülben Ergen','Selda Alkor','Pınar Altuğ','Sanem Çelik','Bergüzar Korel','Bennu Yıldırımlar','Beren Saat','Ayça Bingöl','Meryem Uzerli','Serenay Sarıkaya','Burçin Terzioğlu','Aslı Enver','Özge Özpirinçci','Ebru Şahin','Ezgi Mola','Merve Dizdar','Pınar Deniz']

with open('directors.txt', 'w') as dosya:
    for director in award_winnnig_director:
        dosya.write(director + '\n')
        
with open('actors.txt', 'w') as dosya:
    for actor in award_winning_actors:
        dosya.write(actor + '\n')
        

In [ ]:
#yönetmenlerin ortalama sezon sayıları alınıyor
director_effect = df.groupby('Yönetmen')['Sezon Sayısı'].mean().to_dict()
df['Yönetmen Etkisi'] = df['Yönetmen'].map(director_effect)